## Create Azure Kubernetes Service
Before starting this section, make sure you have set up the global variables from the previous section. Once your global variables have been set up, you no longer have to set them up again, as they are stored in a .env environment file. 

If you delete your .env file or your Azure login session expires, you may need to re-run the setup process as shown in the [previous section](setup_environment.ipynb). Keep this in mind when proceeding onto the other sections in this sample.

### Get Global Variables
First, we will read the previously stored global variables.

In [ ]:
from env_variables import *

### Add AKS Preview CLI Extension

In [ ]:
!az extension add --name aks-preview

### Deploy AKS Cluster & ADD GPU Nodes to run GPU Based Containers
The command below allows you to create an AKS cluster in Azure cloud. This might take some time - please don't cancel the running command. To create an AKS cluster in Azure public MEC, please refer to [create_aks_in_public_mec.md](create_aks_in_public_mec.md). AKS in both Azure cloud and public MEC shares the same configuration steps below.

In [ ]:
!az aks create  --name $aksclustername --resource-group $resourceGroupName --node-resource-group $noderesourceGroupName  --node-count 1 --node-vm-size $gpuvmtype --enable-addons monitoring --generate-ssh-keys

### Connect to the cluster
To manage a Kubernetes cluster, use the Kubernetes command-line client, kubectl.

In [ ]:
!az aks install-cli

### Configure kubectl to connect to your Kubernetes cluster using the az aks get-credentials command. 
The following command downloads credentials and configures the Kubernetes CLI to use them.
Uses ~/.kube/config, the default location for the Kubernetes configuration file. Specify a different location for your Kubernetes configuration file using --file.

In [ ]:
!az aks get-credentials --resource-group $resourceGroupName --name $aksclustername

### Verify the connection to your cluster using the kubectl get command. This command returns a list of the cluster nodes.

In [ ]:
!kubectl get nodes

### Validation Steps

Output of about command should look like this. Make sure the node status is Ready. In case nodes are not ready please restart AKS Cluster.

<img src="../images/_validateaks.JPG" width=400px/>  

### Create an Public IP address and DNS in Azure for AKS cluster so that it can connect with our on-prem edge/kubernetes cluster.

In [ ]:
!az network public-ip create -g  $noderesourceGroupName -n $aksclustername --dns-name $aksdns --allocation-method Static --sku Standard

In [ ]:
akspublicipadress = !az network public-ip show -g $noderesourceGroupName -n $aksclustername --query ipAddress -o tsv
tempVar = set_key(envPath, "AKS_PUBLIC_IP_ADDRESS", akspublicipadress[0].rstrip())

### Enabling GPU Support in Kubernetes
Once your AKS Cluster is setup & GPU nodes are ready , you can enable GPU support by deploying the following Daemonset:
Expected Output of below command > daemonset.apps/nvidia-device-plugin-daemonset created

In [ ]:
!kubectl create -f https://raw.githubusercontent.com/NVIDIA/k8s-device-plugin/v0.9.0/nvidia-device-plugin.yml

### Validation output of below command should have accelerator=nvidia
accelerator=nvidia

<img src="../images/_gpu-node-validation.JPG" width=1000px/> 

In [ ]:
!kubectl get nodes --show-labels

### Mount Azure File Share on Kubernetes Below instructions allow you to mount Azure File Share, 
Microsoft's cloud file system, on your local Kubernetes cluster. It is optional but useful especially when you want to examine file output generated by the video ML containers. In the example below, we will mount an Azure File Share with pre-trained DNN models on Triton container on the AKS cluster.

In [ ]:
!az storage account keys list -g $resourceGroupName -n $storageServiceName
#Retrive the value of key1 and update the value below for both the cells 

FYI -
<img src="../images/update-storage-key.png" width=1000px/>

In [ ]:
#Create a fileshare to host pre-trained DNN models
!az storage share create --account-name $storageServiceName --account-key "Update this value with your storage account key" --name evsfs

In [ ]:
#Create a fileshare to see output frames 
!az storage share create --account-name $storageServiceName --account-key "Update this value with your storage account key" --name evs

In [ ]:
#create output directory in above fileshare to see output frames
!az storage directory create --name output --share-name evs --account-key "Update this value with your storage account key" --account-name $storageServiceName

### Create a storage access secret
Obtain an Microsoft Azure storage account and extract the storage account name (which you provided) and one of the storage account keys. You will then need to create a Kubernetes secret which holds both the account name and key. You can use kubectl directly to create the secret:

In [ ]:
#Please replace the value of <...>(azurestorageaccountkey) with your storageaccount access key 
!kubectl create secret generic azure-fileshare-secret  --from-literal=azurestorageaccountname=$storageServiceName --from-literal=azurestorageaccountkey=<...>


This section completes here - Please proceed to Deploy and Orchestrate Video Pipeline (5.1) in a Two-layer Edge Hierarchy in Readme section 